# Naive Bayes - SMS SPAM Detection

In [1]:
import pandas as pd

In [5]:
df = pd.read_csv("../data/sms_spam_collection.csv",
                 delimiter = "\t",
                 header = None,
                 names=["y", "sms"])
df.head()

,y,sms
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
df.shape

(5572, 2)

In [43]:
df.groupby(['y']).count().reset_index()

,y,sms
0,ham,4825
1,spam,747


In [56]:
p_spam = 747 / 5572
p_ham = 4825 / 5572

print(p_spam, p_ham)

0.13406317300789664 0.8659368269921034


In [57]:
df_p = df.groupby(['y']).count().reset_index()
p_spam = df_p.loc[df_p["y"] == 'spam', 'sms'].values[0] / df.shape[0]
p_ham = df_p.loc[df_p["y"] == 'ham', 'sms'].values[0] / df.shape[0]

print(p_spam, p_ham)

0.13406317300789664 0.8659368269921034


In [12]:
df['sms'].str.split().explode().drop_duplicates()

0                 Go
0              until
0             jurong
0             point,
0            crazy..
            ...     
5569           Pity,
5569        So...any
5569    suggestions?
5570        bitching
5571           Rofl.
Name: sms, Length: 15691, dtype: object

In [60]:
df_p = df.set_index('y')['sms'].str.split().explode().reset_index().groupby(['sms', 'y']).size().to_frame()
df_p = df_p.reset_index()
df_p

,sms,y,0
0,!,ham,48
1,!,spam,8
2,!!,ham,13
3,!!!,ham,5
4,!!!,spam,1
...,...,...,...
16898,“,spam,1
16899,“Harry,spam,1
16900,…,ham,11
16901,….,ham,1


In [61]:
df.set_index('y')['sms'].str.split().explode().reset_index()

,y,sms
0,ham,Go
1,ham,until
2,ham,jurong
3,ham,"point,"
4,ham,crazy..
...,...,...
86904,ham,Its
86905,ham,true
86906,ham,to
86907,ham,its


In [62]:
df_p.describe()

,0
count,16903.000000
mean,5.141632
std,31.833171
min,1.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,1538.000000


In [65]:
df_p[df_p[0] == 1538]

,sms,y,0
15437,to,ham,1538


In [38]:
df_p[df_p["sms"] == 'CALL']

,sms,y,0
1989,CALL,ham,7
1990,CALL,spam,21
